# sqlite3
Python에서는 sqlite3 라이브러리를 기본적으로 제공합니다.

In [1]:
import sqlite3

## 연결 생성
sqlite3 모듈을 사용하기 위해서는 먼저 Connection 객체를 생성하여 데이터베이스에 연결해야합니다.


In [2]:
# 원하는 파일명 지정 (존재하지 않는 파일인경우 생성됨)
con = sqlite3.connect('./database/example.db')

# 실제 파일 경로 대신 :memory: 를 넘기면 RAM에 임시로 데이터베이스가 생성됨
# con = sqlite3.connect(':memory:')

## SQL 실행
Connection 객체를 생성한 다음 Cursor 오브젝트를 생성하여 SQL문을 실행할 수 있도록 합니다.

In [3]:
# Cursor 생성
cur = con.cursor()

# 테이블이 이미 있다면 삭제
cur.execute("DROP TABLE IF EXISTS student")

# 테이블 생성
cur.execute("""
    CREATE TABLE student(
        first_name text,
        last_name text,
        age integer
    )
    """)

# 데이터 삽입
# execute 함수는 한번에 하나의 SQL문만 실행 가능
cur.execute("INSERT INTO student VALUES ('Tomas', 'Train', 10)")
cur.execute("INSERT INTO student VALUES ('Bean', 'Green', 1)")

In [4]:
# 한번에 여러개를 실행하려고 하면 에러 발생
cur.execute("""
            INSERT INTO student VALUES ('Tomas', 'Train', 10);
            INSERT INTO student VALUES ('Bean', 'Green', 1);
            """)

Warning: You can only execute one statement at a time.

In [5]:
# 한번에 여러 SQL문을 실행하고 싶은 경우 executescript 사용하기
cur.executescript("""
    DROP TABLE IF EXISTS teacher;
    DROP TABLE IF EXISTS book;

    CREATE TABLE teacher(
        firstname,
        lastname,
        age
    );

    CREATE TABLE book(
        title,
        author,
        published
    );

    INSERT INTO book(title, author, published)
    VALUES (
        'Dirk Gently''s Holistic Detective Agency',
        'Douglas Adams',
        1987
    );
    """)

In [6]:
# 같은 형식의 SQL문에 데이터만 바꿔서 여러번 실행하고 싶은 경우,
# executemany와 제너레이터를 조합하면 좋다
# 메모리에 모두 올라가는 적은 양의 데이터라면 그냥 리스트로 넘겨도 될듯
def data_generator():
    datas = [
                ['Bee', 'Honey', 3],
                ['Bee', 'Zig', 5], 
                ['Bob', 'Sponge', 7]
            ]
    for data in datas:
        yield data

cur.executemany("INSERT INTO student VALUES (?, ?, ?)", data_generator())
cur.execute("SELECT * FROM student")
cur.fetchall()

[('Tomas', 'Train', 10),
 ('Bean', 'Green', 1),
 ('Bee', 'Honey', 3),
 ('Bee', 'Zig', 5),
 ('Bob', 'Sponge', 7)]

### 결과 조회
SQL 결과를 조회하기 위해서는 Cursor 객체를 iterator로 활용하는 방법,  
fetchone, fetchmany, fetchall 함수를 이용하는 방법이 있습니다.


In [7]:
# Cursor는 iterator객체이므로 순차적으로 값을 불러올 수 있다
cur.execute("SELECT * FROM student").rowcount

print(next(cur))
print(next(cur))

('Tomas', 'Train', 10)
('Bean', 'Green', 1)


In [8]:
# SQL 실행하면 동일한 cursor 객체를 리턴해준다
cur.execute("SELECT * FROM student") is cur

True

In [9]:
# 그래서 이렇게 바로 for문에 넣는것도 가능!
for row in cur.execute("SELECT * FROM student"):
    print(row)

('Tomas', 'Train', 10)
('Bean', 'Green', 1)
('Bee', 'Honey', 3)
('Bee', 'Zig', 5)
('Bob', 'Sponge', 7)


fetchone, fetchmany, fetchall 함수를 이용하여 결과를 가져올 경우,  
이미 조회한 데이터는 다시 함수를 호출해도 조회할 수 없습니다. (다음 데이터부터 가져옴)  
다시 조회하고 싶다면 SQL문을 다시 실행해야합니다.

In [12]:
# fetchone은 실행결과 중 하나의 행만 가져온다
cur.execute("SELECT * FROM student")
cur.fetchone()

('Tomas', 'Train', 10)

In [13]:
# 한번 더 호출해보면 그 다음 행을 가져온다
cur.fetchone()

('Bean', 'Green', 1)

In [14]:
# fetchmany는 실행 결과 중 원하는 개수의 행을 가져올 수 있다
# 원하는 개수보다 적은 행이 남아있으면 남은 행만큼만 가져온다
cur.fetchmany(5)

[('Bee', 'Honey', 3), ('Bee', 'Zig', 5), ('Bob', 'Sponge', 7)]

In [15]:
# fetchall은 실행 결과 중 남아있는 모든 행을 가져온다
# 남아있는 행이 없으면 빈 리스트 리턴
cur.fetchall()

[]

In [16]:
# 다시 조회하고 싶으면 SQL문을 다시 실행해야한다
cur.execute("SELECT * FROM student")
cur.fetchall()

[('Tomas', 'Train', 10),
 ('Bean', 'Green', 1),
 ('Bee', 'Honey', 3),
 ('Bee', 'Zig', 5),
 ('Bob', 'Sponge', 7)]

### 변경사항 저장
Connection객체의 commit함수로 변경사항을 저장할 수 있습니다.

In [17]:
# commit으로 변경사항 저장
con.commit()

# 다시 연결한 다음 확인해보기
con.close()

con = sqlite3.connect('./database/example.db')
cur = con.cursor()

# 변경사항이 저장됐다
for row in cur.execute("SELECT * FROM student"):
    print(row)

('Tomas', 'Train', 10)
('Bean', 'Green', 1)
('Bee', 'Honey', 3)
('Bee', 'Zig', 5)
('Bob', 'Sponge', 7)


## 테이블 보기 좋게 출력하기

### prettytable
이번 프로젝트에서는 prettytable 모듈을 활용했습니다.  
pandas를 활용해도 좋겠지만 단순히 SQL 실행 결과만 확인할 것이기 때문에 prettytable로도 충분합니다.

In [20]:
# prettytable 모듈 버전 확인
!pip show prettytable | findstr Version

# 리눅스 환경에서는 findstr 말고 grep 사용하기
# !pip show prettytable | grep Version

# 모듈 없는 경우 주석 해제하고 설치하기
# !pip install prettytable

Version: 3.9.0


Python DB-API를 통해 지원되는 DB모듈의 경우 Cursor 객체를 바로 테이블로 만들 수 있습니다

In [21]:
from prettytable import from_db_cursor

cur.execute("SELECT * FROM student")
from_db_cursor(cur)

first_name,last_name,age
Tomas,Train,10
Bean,Green,1
Bee,Honey,3
Bee,Zig,5
Bob,Sponge,7


### 스키마
sqlite에서 스키마를 확인하려면 DESC 명령어 대신 다른 방법을 사용해야 합니다.  
https://www.sqlitetutorial.net/sqlite-describe-table/

* sqlite_schema 테이블에 저장된 SQL 확인하기

In [31]:
cur.execute(
    """
    SELECT *
    FROM sqlite_schema 
    WHERE name = 'student';
    """
            )
from_db_cursor(cur)

type,name,tbl_name,rootpage,sql
table,student,student,2,"CREATE TABLE student( first_name text, last_name text, age integer )"


In [33]:
# sqlite_master 테이블로도 동일하게 스키마 확인 가능 (대체 가능한 별칭)
cur.execute(
    """
    SELECT *
    FROM sqlite_master 
    WHERE name = 'student';
    """
            )
from_db_cursor(cur)

type,name,tbl_name,rootpage,sql
table,student,student,2,"CREATE TABLE student( first_name text, last_name text, age integer )"


In [36]:
# 모든 테이블의 스키마도 확인 가능
cur.execute("SELECT * FROM sqlite_master")
from_db_cursor(cur)

type,name,tbl_name,rootpage,sql
table,student,student,2,"CREATE TABLE student( first_name text, last_name text, age integer )"
table,teacher,teacher,4,"CREATE TABLE teacher( firstname, lastname, age )"
table,book,book,3,"CREATE TABLE book( title, author, published )"



* PARAGMA table_info(테이블명) 명령문으로 스키마 확인하기

In [37]:
cur.execute("PRAGMA table_info('student')")
from_db_cursor(cur)

cid,name,type,notnull,dflt_value,pk
0,first_name,TEXT,0,None,0
1,last_name,TEXT,0,None,0
2,age,INTEGER,0,None,0
